# <img style="float: left; padding-right: 10px; width: 45px" src="https://github.com/Harvard-IACS/2018-CS109A/blob/master/content/styles/iacs.png?raw=true"> CS109A Introduction to Data Science 

## Lab 2 Pandas

**Harvard University**<br>
**Summer 2018**<br>
**Instructors:** Pavlos Protopapas and Kevin Rader <br>
**Lab Instructors:** Rahul Dave <br>
**Authors:** Rahul Dave, David Sondak, Will Claybaugh and Pavlos Protopaps

---

In [1]:
## RUN THIS CELL TO GET THE RIGHT FORMATTING 
from IPython.core.display import HTML
def css_styling():
    styles = open("../../styles/cs109.css", "r").read()
    return HTML(styles)
css_styling()

# Table of Contents 
<ol start="0">
<li> Learning Goals </li>
<li> Loading and Cleaning with Pandas</li>
<li> Parsing and Completing the Dataframe  </li>
<li> Grouping </li>
</ol>

## Learning Goals

About 6000 odd "best books" were fetched and parsed from [Goodreads](https://www.goodreads.com). The "bestness" of these books came from a proprietary formula used by Goodreads and published as a list on their web site.

We parsed the page for each book and saved data from all these pages in a tabular format as a CSV file. In this lab we'll clean and further parse the data.  We'll then do some exploratory data analysis to answer questions about these best books and popular genres.  


By the end of this lab, you should be able to:

- Load and systematically address missing values, ancoded as `NaN` values in our data set, for example, by removing observations associated with these values.
- Parse columns in the dataframe to create new dataframe columns.
- Use groupby to aggregate data on a particular feature column, such as author.

*This lab corresponds to lectures 2, 3 and 4 and maps on to homework 1 and further.*

### Basic EDA workflow

(From the lecture, repeated here for convenience).

The basic workflow is as follows:

1. **Build** a DataFrame from the data (ideally, put all data in this object)
2. **Clean** the DataFrame. It should have the following properties:
    - Each row describes a single object
    - Each column describes a property of that object
    - Columns are numeric whenever appropriate
    - Columns contain atomic properties that cannot be further decomposed
3. Explore **global properties**. Use histograms, scatter plots, and aggregation functions to summarize the data.
4. Explore **group properties**. Use groupby and small multiples to compare subsets of the data.

This process transforms your data into a format which is easier to work with, gives you a basic overview of the data's properties, and likely generates several questions for you to followup in subsequent analysis.

## Part 1: Loading and Cleaning with Pandas 
Read in the `goodreads.csv` file, examine the data, and do any necessary data cleaning. 

Here is a description of the columns (in order) present in this csv file:

```
rating: the average rating on a 1-5 scale achieved by the book
review_count: the number of Goodreads users who reviewed this book
isbn: the ISBN code for the book
booktype: an internal Goodreads identifier for the book
author_url: the Goodreads (relative) URL for the author of the book
year: the year the book was published
genre_urls: a string with '|' separated relative URLS of Goodreads genre pages
dir: a directory identifier internal to the scraping code
rating_count: the number of ratings for this book (this is different from the number of reviews)
name: the name of the book
```

Let us see what issues we find with the data and resolve them.  



----




After loading appropriate libraries


In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Cleaning: Reading in the data
We read in and clean the data from `goodreads.csv`.

In [7]:
#Read the data into a dataframe
df = pd.read_csv("data/goodreads.csv", header = None)


#Examine the first few rows of the dataframe
df.head()

,0,1,2,3,4,5,6,7,8,9
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008.0,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice


Oh dear. That does not quite seem to be right. We are missing the column names. We need to add these in! But what are they?

Here is a list of them in order:

`["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name']`

<div class="exercise"><b>Exercise</b></div>
Use these to load the dataframe properly! And then "head" the dataframe... (you will need to look at the read_csv docs)


In [12]:
# your code here
df = pd.read_csv("data/goodreads.csv", 
                 names = ["rating", 'review_count', 'isbn', 'booktype','author_url', 'year', 'genre_urls', 'dir','rating_count', 'name'])
df.head()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
0,4.40,136455,0439023483,good_reads:book,https://www.goodreads.com/author/show/153394.S...,2008.0,/genres/young-adult|/genres/science-fiction|/g...,dir01/2767052-the-hunger-games.html,2958974,"The Hunger Games (The Hunger Games, #1)"
1,4.41,16648,0439358078,good_reads:book,https://www.goodreads.com/author/show/1077326....,2003.0,/genres/fantasy|/genres/young-adult|/genres/fi...,dir01/2.Harry_Potter_and_the_Order_of_the_Phoe...,1284478,Harry Potter and the Order of the Phoenix (Har...
2,3.56,85746,0316015849,good_reads:book,https://www.goodreads.com/author/show/941441.S...,2005.0,/genres/young-adult|/genres/fantasy|/genres/ro...,dir01/41865.Twilight.html,2579564,"Twilight (Twilight, #1)"
3,4.23,47906,0061120081,good_reads:book,https://www.goodreads.com/author/show/1825.Har...,1960.0,/genres/classics|/genres/fiction|/genres/histo...,dir01/2657.To_Kill_a_Mockingbird.html,2078123,To Kill a Mockingbird
4,4.23,34772,0679783261,good_reads:book,https://www.goodreads.com/author/show/1265.Jan...,1813.0,/genres/classics|/genres/fiction|/genres/roman...,dir01/1885.Pride_and_Prejudice.html,1388992,Pride and Prejudice


### Cleaning: Examing the dataframe - quick checks

We should examine the dataframe to get a overall sense of the content. 

<div class="exercise"><b>Exercise</b></div>
Lets check the types of the columns. What do you find?

In [13]:
# your code here
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

*your answer here*


There are a couple more quick sanity checks to perform on the dataframe. 

In [15]:

print(df.shape)
df.columns

(6000, 10)


Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name'], dtype='object')

In [14]:
! wc -l data/goodreads.csv

    6000 data/goodreads.csv


### Cleaning: Examining the dataframe - a deeper look

Beyond performing checking some quick general properties of the data frame and looking at the first $n$ rows, we can dig a bit deeper into the values being stored. If you haven't already, check to see if there are any missing values in the data frame.

Let's see for a column which seemed OK to us.

In [16]:
#Get a sense of how many missing values there are in the dataframe.
np.sum([df.rating.isnull()])

2

In [17]:
#Try to locate where the missing values occur
df[df.rating.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
3643,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5282,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None


How does `pandas` or `numpy` handle missing values when we try to compute with data sets that include them?

We'll now check if any of the other suspicious columns have missing values.  Let's look at `year` and `review_count` first.

One thing you can do is to try and convert to the type you expect the column to be. If something goes wrong, it likely means your data are bad.

Lets test for missing data:

In [18]:
df[df.year.isnull()]

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
2442,4.23,526,NaN,good_reads:book,https://www.goodreads.com/author/show/623606.A...,NaN,/genres/religion|/genres/islam|/genres/non-fic...,dir25/1301625.La_Tahzan.html,4134,La Tahzan
2869,4.61,2,NaN,good_reads:book,https://www.goodreads.com/author/show/8182217....,NaN,NaN,dir29/22031070-my-death-experiences---a-preach...,23,My Death Experiences - A Preacherâs 18 Apoca...
3643,NaN,None,None,None,None,NaN,NaN,dir37/9658936-harry-potter.html,None,None
5282,NaN,None,None,None,None,NaN,NaN,dir53/113138.The_Winner.html,None,None
5572,3.71,35,8423336603,good_reads:book,https://www.goodreads.com/author/show/285658.E...,NaN,/genres/fiction,dir56/890680._rase_una_vez_el_amor_pero_tuve_q...,403,Ãrase una vez el amor pero tuve que matarlo. ...
5658,4.32,44,NaN,good_reads:book,https://www.goodreads.com/author/show/25307.Ro...,NaN,/genres/fantasy|/genres/fantasy|/genres/epic-f...,dir57/5533041-assassin-s-apprentice-royal-assa...,3850,Assassin's Apprentice / Royal Assassin (Farsee...
5683,4.56,204,NaN,good_reads:book,https://www.goodreads.com/author/show/3097905....,NaN,/genres/fantasy|/genres/young-adult|/genres/ro...,dir57/12474623-tiger-s-dream.html,895,"Tiger's Dream (The Tiger Saga, #5)"


### Cleaning: Dealing with Missing Values
How should we interpret 'missing' or 'invalid' values in the data (hint: look at where these values occur)? One approach is to simply exclude them from the dataframe. Is this appropriate for all 'missing' or 'invalid' values? 

In [19]:
#Treat the missing or invalid values in your dataframe
####### 
df.shape
df = df[df.year.notnull()] # removing NULL rows
df.shape

(5993, 10)

Ok so we have done some cleaning. What do things look like now? Notice the float has not yet changed.

In [20]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year            float64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

In [21]:
print(np.sum(df.year.isnull()))
print(np.sum(df.rating_count.isnull())) 
print(np.sum(df.review_count.isnull())) 
# We removed seven rows
df.shape

0
0
0


(5993, 10)

*Suspect observations for rating and rating_count were removed as well! *

<div class="exercise"><b>Exercise</b></div>

Ok so lets fix those types. Convert them to ints. If the type conversion fails, we now know we have further problems.

In [25]:
# your code here
df['year'] = df["year"].astype(int)
df['rating_count'] = df["rating_count"].astype(int)
df['review_count'] = df["review_count"].astype(int)

Once you do this, we seem to be good on these columns (no errors in conversion). Lets look:

In [24]:
df.dtypes

rating          float64
review_count     object
isbn             object
booktype         object
author_url       object
year              int64
genre_urls       object
dir              object
rating_count     object
name             object
dtype: object

Sweet!

Some of the other colums that should be strings have NaN.

In [15]:
df.loc[df.genre_urls.isnull(), 'genre_urls']=""
df.loc[df.isbn.isnull(), 'isbn']=""

##  Part 2: Parsing and Completing the Data Frame 

We will parse the `author` column from the author_url and `genres` column from the genre_urls. Keep the `genres` column as a string separated by '|'.

We will use panda's `map` to assign new columns to the dataframe.  

Examine an example `author_url` and reason about which sequence of string operations must be performed in order to isolate the author's name.

In [26]:
#Get the first author_url
test_string = df.author_url[0]
test_string

'https://www.goodreads.com/author/show/153394.Suzanne_Collins'

In [30]:
#Test out some string operations to isolate the author name
#[1:] remove the first element (in case author name has .)
test_string.split('/')[-1].split('.')[1:][0]

['Suzanne_Collins']

<div class="exercise"><b>Exercise</b></div>

Lets wrap the above code into a function which we will then use

In [32]:
# Write a function that accepts an author url and returns the author's name based on your experimentation above
def get_author(url):
    # your code here
    name = url.split('/')[-1].split('.')[1:][0]
    ####### 
    
    return name

In [33]:
#Apply the get_author function to the 'author_url' column using '.map' 
#and add a new column 'author' to store the names
df['author'] = df.author_url.map(get_author)
df.author[0:5]

0    Suzanne_Collins
1        J_K_Rowling
2    Stephenie_Meyer
3         Harper_Lee
4        Jane_Austen
Name: author, dtype: object

<div class="exercise"><b>Exercise</b></div>

Now parse out the genres from `genre_url`.  

This is a little more complicated because there be more than one genre.


In [20]:

df.genre_urls.head()

In [21]:
# your code here


<div class="exercise"><b>Exercise</b></div>

Write a function that accepts a genre url and returns the genre name based on your experimentation above



In [22]:
def split_and_join_genres(url):
    # your code here


Test your function

In [23]:
split_and_join_genres("/genres/young-adult|/genres/science-fiction")

In [24]:
split_and_join_genres("")

<div class="exercise"><b>Exercise</b></div>

Use map again to create a new "genres" column

In [25]:

df['genres']=df.genre_urls.map(split_and_join_genres)
df.head()

Finally, let's pick an author at random so we can see the results of the transformations.  Scroll to see the `author` and `genre` columns that we added to the dataframe.

In [26]:
df[df.author == "Marguerite_Yourcenar"]

Let us delete the `genre_urls` column.

In [27]:
del df['genre_urls']

And then save the dataframe out!

In [28]:
df.to_csv("data/cleaned-goodreads.csv", index=False, header=True)

---

## Part 3: Grouping 

It appears that some books were written in negative years!  Print out the observations that correspond to negative years.  What do you notice about these books?  

In [29]:
# your code here


We can determine the "best book" by year! For this we use Pandas groupby. Groupby allows grouping a dataframe by any (usually categorical) variable.

In [34]:
dfgb_author = df.groupby('author')
type(dfgb_author)

pandas.core.groupby.groupby.DataFrameGroupBy

In [36]:
df.columns

Index(['rating', 'review_count', 'isbn', 'booktype', 'author_url', 'year', 'genre_urls', 'dir', 'rating_count', 'name', 'author'], dtype='object')

Perhaps we want the number of books each author wrote

In [35]:
dfgb_author.count()

,rating,review_count,isbn,booktype,author_url,year,genre_urls,dir,rating_count,name
author,,,,,,,,,,
A_A_Milne,6,6,6,6,6,6,6,6,6,6
A_G_Howard,1,1,1,1,1,1,1,1,1,1
A_J_Cronin,1,1,1,1,1,1,1,1,1,1
A_J_Jacobs,1,1,1,1,1,1,1,1,1,1
A_J_Salt,1,1,0,1,1,1,0,1,1,1
A_Meredith_Walters,2,2,0,2,2,2,2,2,2,2
A_N_Roquelaure,2,2,2,2,2,2,2,2,2,2
A_S_Byatt,1,1,1,1,1,1,1,1,1,1
A_S_King,1,1,1,1,1,1,1,1,1,1


Lots of useless info there. One column should suffice

In [37]:
dfgb_author['author'].count()

author
A_A_Milne                 6
A_G_Howard                1
A_J_Cronin                1
A_J_Jacobs                1
A_J_Salt                  1
A_Meredith_Walters        2
A_N_Roquelaure            2
A_S_Byatt                 1
A_S_King                  1
A_id_al_Qarni             2
Abbi_Glines              14
Abdul_Rahman_Munif        1
Abigail_Gibbs             1
Abigail_Roux              4
Abigail_Thomas            1
Abolqasem_Ferdowsi        1
Abraham_Verghese          1
Abul_Hasan_Ali_Nadwi      1
Adam_Hochschild           1
Adam_Johnson              1
Adam_Levin                1
Adam_Rex                  1
Adam_Smith                1
Addison_Moore             1
Adeline_Yen_Mah           1
Adolf_Hitler              1
Adolfo_Bioy_Casares       1
Aeschylus                 1
Aesop                     1
Agatha_Christie          11
                         ..
William_Strunk_Jr_        1
William_Styron            3
William_Wharton           2
William_Wordsworth        1
Willow_Aster 

Perhaps you want more detailed info...

In [38]:
dfgb_author[['rating', 'rating_count', 'review_count', 'year']].describe()

rating                                                        rating_count                                                                                   review_count                                                                             year                                                                   
                       count      mean       std   min     25%    50%     75%   max        count           mean            std       min        25%       50%        75%       max        count          mean           std      min       25%      50%       75%      max count         mean         std     min      25%     50%      75%     max
author                                                                                                                                                                                                                                                                                                                                             
A_A_Milne                6.0  4.365000  0.078677  4.25  4.3275  4.365  4.4025  4.48          6.0   47842.333333   57135.314010     544.0   15559.50   30547.0   50401.25  157833.0          6.0    543.000000    682.473150      1.0    219.50    309.5    499.25   1886.0   6.0  1944.166667   29.294482  1926.0  1926.25  1927.5  1952.75  1997.0
A_G_Howard               1.0  4.020000       NaN  4.02  4.0200  4.020  4.0200  4.02          1.0   17073.000000            NaN   17073.0   17073.00   17073.0   17073.00   17073.0          1.0   3194.000000           NaN   3194.0   3194.00   3194.0   3194.00   3194.0   1.0  2013.000000         NaN  2013.0  2013.00  2013.0  2013.00  2013.0
A_J_Cronin               1.0  4.220000       NaN  4.22  4.2200  4.220  4.2200  4.22          1.0    1015.000000            NaN    1015.0    1015.00    1015.0    1015.00    1015.0          1.0    109.000000           NaN    109.0    109.00    109.0    109.00    109.0   1.0  1941.000000         NaN  1941.0  1941.00  1941.0  1941.00  1941.0
A_J_Jacobs               1.0  3.750000       NaN  3.75  3.7500  3.750  3.7500  3.75          1.0   39489.000000            NaN   39489.0   39489.00   39489.0   39489.00   39489.0          1.0   4371.000000           NaN   4371.0   4371.00   4371.0   4371.00   4371.0   1.0  2007.000000         NaN  2007.0  2007.00  2007.0  2007.00  2007.0
A_J_Salt                 1.0  4.940000       NaN  4.94  4.9400  4.940  4.9400  4.94          1.0      16.000000            NaN      16.0      16.00      16.0      16.00      16.0          1.0      6.000000           NaN      6.0      6.00      6.0      6.00      6.0   1.0  2014.000000         NaN  2014.0  2014.00  2014.0  2014.00  2014.0
A_Meredith_Walters       2.0  4.150000  0.169706  4.03  4.0900  4.150  4.2100  4.27          2.0   16391.500000    5562.809048   12458.0   14424.75   16391.5   18358.25   20325.0          2.0   1417.500000    354.260497   1167.0   1292.25   1417.5   1542.75   1668.0   2.0  2012.500000    0.707107  2012.0  2012.25  2012.5  2012.75  2013.0
A_N_Roquelaure           2.0  3.450000  0.226274  3.29  3.3700  3.450  3.5300  3.61          2.0   25967.000000    9480.887722   19263.0   22615.00   25967.0   29319.00   32671.0          2.0   1796.000000   1633.416665    641.0   1218.50   1796.0   2373.50   2951.0   2.0  1983.500000    0.707107  1983.0  1983.25  1983.5  1983.75  1984.0
A_S_Byatt                1.0  3.860000       NaN  3.86  3.8600  3.860  3.8600  3.86          1.0   45361.000000            NaN   45361.0   45361.00   45361.0   45361.00   45361.0          1.0   2761.000000           NaN   2761.0   2761.00   2761.0   2761.00   2761.0   1.0  1990.000000         NaN  1990.0  1990.00  1990.0  1990.00  1990.0
A_S_King                 1.0  3.930000       NaN  3.93  3.9300  3.930  3.9300  3.93          1.0   16097.000000            NaN   16097.0   16097.00   16097.0   16097.00   16097.0          1.0   2108.000000           NaN   2108.0   2108.00   2108.0   2108.00   2108.0   1.0  2010.000

You can also access a `groupby` dictionary style.

In [34]:
ratingdict = {}
for author, subset in dfgb_author:
    ratingdict[author] = (subset['rating'].mean(), subset['rating'].std())
ratingdict

<div class="exercise"><b>Exercise</b></div>

Lets get the best-rated book(s) for every year in our dataframe.

In [35]:
#Using .groupby, we can divide the dataframe into subsets by the values of 'year'.
#We can then iterate over these subsets
# your code here
